In [2]:
# 한글 폰트 사용을 위한 라이브러리입니다.
!apt-get install -y fonts-nanum

zsh:1: command not found: apt-get


In [4]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import BallTree
from sklearn import metrics

from sklearn.compose   import ColumnTransformer
from sklearn.pipeline  import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from category_encoders     import TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error


import eli5
from eli5.sklearn import PermutationImportance

ModuleNotFoundError: No module named 'matplotlib'

In [77]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '../Public/data/train.csv'
test_path  = '../Public/data/test.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)

In [111]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
dt['is_test'] = 0
dt_test['is_test'] = 1
concat = pd.concat([dt, dt_test])     # 하나의 데이터로 만들어줍니다.

In [112]:
# 칼럼 이름을 쉽게 바꿔주겠습니다. 다른 칼럼도 사용에 따라 바꿔주셔도 됩니다!
concat = concat.rename(columns={'전용면적(㎡)':'전용면적'})

In [127]:
# 위 처럼 아무 의미도 갖지 않는 칼럼은 결측치와 같은 역할을 하므로, np.nan으로 채워 결측치로 인식되도록 합니다.
concat['등기신청일자'] = concat['등기신청일자'].replace(' ', np.nan)
concat['거래유형'] = concat['거래유형'].replace('-', np.nan)
concat['중개사소재지'] = concat['중개사소재지'].replace('-', np.nan)

In [128]:
# 위에서 결측치가 100만개 이하인 변수들만 골라 새로운 concat_select 객체로 저장해줍니다.
selected = list(concat.columns[concat.isnull().sum() <= 1000000])
concat_select = concat[selected]

In [129]:
# 본번, 부번의 경우 float로 되어있지만 범주형 변수의 의미를 가지므로 object(string) 형태로 바꾸어주고 아래 작업을 진행하겠습니다.
concat_select['본번'] = concat_select['본번'].astype('str')
concat_select['부번'] = concat_select['부번'].astype('str')

여기서 X, Y 좌표 결측치를  채워넣어야 할 것 같음

In [130]:
# 먼저, 연속형 변수와 범주형 변수를 위 info에 따라 분리해주겠습니다.
continuous_columns = []
categorical_columns = []

for column in concat_select.columns:
    if pd.api.types.is_numeric_dtype(concat_select[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

연속형 변수: ['전용면적', '계약년월', '계약일', '층', '건축년도', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', '건축면적', '주차대수', '좌표X', '좌표Y', 'target', 'is_test']
범주형 변수: ['시군구', '번지', '본번', '부번', '아파트명', '도로명', 'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형', 'k-난방방식', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일']


In [131]:
# 범주형 변수에 대한 보간
concat_select[categorical_columns] = concat_select[categorical_columns].fillna('NULL')

# 연속형 변수에 대한 보간 (선형 보간)
concat_select[continuous_columns] = concat_select[continuous_columns].interpolate(method='linear', axis=0)

In [132]:
# 이상치 제거 방법에는 IQR을 이용하겠습니다.
def remove_outliers_iqr(dt, column_name):
    df = dt.query('is_test == 0')       # train data 내에 있는 이상치만 제거하도록 하겠습니다.
    df_test = dt.query('is_test == 1')

    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

    result = pd.concat([df, df_test])   # test data와 다시 합쳐주겠습니다.
    return result

In [133]:
# 위 방법으로 전용 면적에 대한 이상치를 제거해보겠습니다.
concat_select = remove_outliers_iqr(concat_select, '전용면적')

In [134]:
# 시군구, 년월 등 분할할 수 있는 변수들은 세부사항 고려를 용이하게 하기 위해 모두 분할해 주겠습니다.
concat_select['구'] = concat_select['시군구'].map(lambda x : x.split()[1])
concat_select['동'] = concat_select['시군구'].map(lambda x : x.split()[2])
del concat_select['시군구']

In [135]:
# 강남 여부를 표시하는 피쳐를 생성합니다.

all = list(concat_select['구'].unique())
gangnam = ['강서구', '영등포구', '동작구', '서초구', '강남구', '송파구', '강동구']
gangbuk = [x for x in all if x not in gangnam]

assert len(all) == len(gangnam) + len(gangbuk)       # 알맞게 분리되었는지 체크합니다.

is_gangnam = []
for x in concat_select['구'].tolist() :
  if x in gangnam :
    is_gangnam.append(1)
  else :
    is_gangnam.append(0)

# 파생변수를 하나 만듭니다.
concat_select['강남여부'] = is_gangnam

In [136]:
concat_select.columns

Index(['번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도', '도로명',
       'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식',
       'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사',
       'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-수정일자', '고용보험관리번호',
       '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4',
       '단지승인일', '사용허가여부', '관리비 업로드', '좌표X', '좌표Y', '단지신청일', 'target',
       'is_test', '구', '동', '강남여부'],
      dtype='object')

In [137]:
### 계약년, 계약월 변수 생성 후, 학습 데이터의 최초 기간부터 경과한 기간을 계산합니다.

concat_select['계약년'] = (
    concat_select['계약년월']
    .astype(str)
    .str[:4]
    .astype(int)
)
concat_select['계약월'] = (
    concat_select['계약년월']
    .astype(str)
    .str[4:6]
    .astype(int)
)

concat_select.drop(columns='계약년월', inplace=True)

BASE_YEAR  = 2007
BASE_MONTH = 1

concat_select['거래개월수'] = (
    (concat_select['계약년']  - BASE_YEAR) * 12
  + (concat_select['계약월'] - BASE_MONTH)
)

concat_select['거래개월수'].head()

0    131
1    131
2    131
3    132
4    132
Name: 거래개월수, dtype: int64

In [138]:
### 건축년도를 사용하여 건축연수라는 새로운 피쳐를 생성합니다.

# 1) 연도 계산용 기준 년도 설정
CURRENT_YEAR = 2025

# 2) '건축연수' 파생변수 생성
#    concat_select 혹은 원하는 DataFrame 이름으로 바꿔서 쓰세요.
concat_select['건축연수'] = CURRENT_YEAR - concat_select['건축년도'].astype(int)

# 3) 확인
print(concat_select[['건축년도','건축연수']].head())

   건축년도  건축연수
0  1987    38
1  1987    38
2  1987    38
3  1987    38
4  1987    38


In [139]:
### 부동산 데이터와 패스트푸드점 데이터의 좌표를 사용하여 "반경 500 미터 내에 패스트푸드점 갯수" 피쳐를 생성합니다.

fastfood_branches = pd.read_csv(
    'kakao_burger_all_seoul.csv', encoding='utf-8')

# address 에서 “○○구”, “○○동” 추출하기 (정규식) 첫 번째 매칭되는 “어절+구”, “어절+동” 을 잡아옵니다.
fastfood_branches['구'] = fastfood_branches['address_name'].str.extract(r'(\w+구)')
fastfood_branches['동'] = fastfood_branches['address_name'].str.extract(r'(\w+동)')

fastfood_branches.head(10)

,brand,place_name,address_name,lng,lat,구,동
0,KFC,KFC 역삼역점,서울 강남구 역삼동 642-10,127.036149,37.502004,강남구,역삼동
1,KFC,KFC 압구정로데오점,서울 강남구 신사동 660-5,127.037473,37.528368,강남구,신사동
2,KFC,KFC 강남구청역점,서울 강남구 삼성동 1,127.041921,37.517071,강남구,삼성동
3,KFC,KFC 대치동점,서울 강남구 대치동 987-20,127.062959,37.499769,강남구,대치동
4,KFC,KFC 코엑스몰점,서울 강남구 삼성동 159,127.058660,37.511114,강남구,삼성동
5,KFC,KFC 신사역점,서울 강남구 논현동 5-1,127.023415,37.517422,강남구,논현동
6,KFC,KFC 학동역2점,서울 강남구 논현동 89-22,127.033126,37.514994,강남구,논현동
7,KFC,KFC 일원동점,서울 강남구 일원동 615-1,127.080414,37.491206,강남구,일원동
8,KFC,KFC 양재동점,서울 서초구 양재동 12-3,127.035191,37.484442,서초구,양재동
9,KFC,KFC 서초우성점,서울 서초구 서초동 1330-8,127.028310,37.492669,서초구,서초동


In [140]:
# 1) pick out the two groups of brands
group1 = ['롯데리아', '맘스터치']
group2 = ['맥도날드', 'KFC', '버거킹']

df1 = fastfood_branches[fastfood_branches['brand'].isin(group1)]
df2 = fastfood_branches[fastfood_branches['brand'].isin(group2)]

# 2) build BallTrees (haversine expects lat/lon in radians)
#    we’ll stack as [lat, lon]
br1 = np.deg2rad(df1[['lat','lng']].values)
br2 = np.deg2rad(df2[['lat','lng']].values)

tree1 = BallTree(br1, metric='haversine')
tree2 = BallTree(br2, metric='haversine')

# 3) prepare apartment coords
apt_coords = np.deg2rad(concat_select[['좌표Y','좌표X']].values)

# 4) query radius = 1km → radians on earth
earth_r = 6_371_000  # metres
rad = 1_000 / earth_r

idxs1 = tree1.query_radius(apt_coords, r=rad)
idxs2 = tree2.query_radius(apt_coords, r=rad)

# 5) count and assign
concat_select['Lot_Mst_within_1km']    = [len(idx) for idx in idxs1]
concat_select['Mc_KFC_BK_within_1km'] = [len(idx) for idx in idxs2]

print(concat_select.head(10))

      번지     본번   부번    아파트명   전용면적  계약일  층  건축년도    도로명 k-단지분류(아파트,주상복합등등)  \
0  658-1  658.0  1.0  개포6차우성  79.97    8  3  1987  언주로 3                아파트   
1  658-1  658.0  1.0  개포6차우성  79.97   22  4  1987  언주로 3                아파트   
2  658-1  658.0  1.0  개포6차우성  54.98   28  5  1987  언주로 3                아파트   
3  658-1  658.0  1.0  개포6차우성  79.97    3  4  1987  언주로 3                아파트   
4  658-1  658.0  1.0  개포6차우성  79.97    8  2  1987  언주로 3                아파트   
5  658-1  658.0  1.0  개포6차우성  79.97   11  1  1987  언주로 3                아파트   
6  658-1  658.0  1.0  개포6차우성  79.97   19  2  1987  언주로 3                아파트   
7  658-1  658.0  1.0  개포6차우성  54.98    5  5  1987  언주로 3                아파트   
8  658-1  658.0  1.0  개포6차우성  79.97   28  3  1987  언주로 3                아파트   
9  658-1  658.0  1.0  개포6차우성  54.98    9  3  1987  언주로 3                아파트   

   ... is_test    구    동 강남여부   계약년 계약월  거래개월수  건축연수 Lot_Mst_within_1km  \
0  ...       0  강남구  개포동    1  2017  12    131    38   

In [141]:
### 주변 중학교의 학업성취도 관련한 피쳐를 생성합니다. 
# 1) 중학교 데이터 로드
df_sch = pd.read_csv(
    'middle_schools_with_coords_and_roadaddr.csv',
    encoding='utf-8-sig'
).dropna(subset=['학업성취도','X좌표(경도)','Y좌표(위도)'])

# float 변환
df_sch['X좌표(경도)'] = df_sch['X좌표(경도)'].astype(float)
df_sch['Y좌표(위도)'] = df_sch['Y좌표(위도)'].astype(float)

# 2) BallTree 준비 (haversine expects [lat, lon] in radians)
school_coords = np.deg2rad(df_sch[['Y좌표(위도)','X좌표(경도)']].values)
school_achv   = df_sch['학업성취도'].values
tree = BallTree(school_coords, metric='haversine')

# 3) 아파트 좌표 준비
#    concat_select 에 '좌표X','좌표Y' 가 float 형으로 있어야 합니다.
concat_select['좌표X'] = concat_select['좌표X'].astype(float)
concat_select['좌표Y'] = concat_select['좌표Y'].astype(float)
apt_coords = np.deg2rad(concat_select[['좌표Y','좌표X']].values)

# 4) 반경 설정: 2km → radians
earth_r = 6_371_000
radius  = 2_000 / earth_r   #도시(서울·수도권 등) 일반 공립 중학교 대부분 거주지 기준 ‘1~3km 이내’ 학군으로 묶입니다. 

# 5) 피쳐 저장용 리스트
mean_achv     = []
max_achv      = []
count_schools = []
wmean_achv    = []

# 6) 아파트 한 건씩 쿼리
for coord in apt_coords:
    # coord: shape (2,) → reshape to (1,2) for query_radius
    inds, dists = tree.query_radius(coord.reshape(1,-1), 
                                    r=radius, 
                                    return_distance=True)
    inds = inds[0]
    dists = dists[0]
    if inds.size == 0:
        mean_achv.append(np.nan)
        max_achv.append(np.nan)
        count_schools.append(0)
        wmean_achv.append(np.nan)
    else:
        achvs = school_achv[inds]
        mean_achv.append(achvs.mean())
        max_achv.append(achvs.max())
        count_schools.append(len(inds))
        # 거리 가중치 = 1/(거리+ε)
        w = 1.0 / (dists + 1e-6)
        wmean_achv.append((achvs * w).sum() / w.sum())

# 7) concat_select 에 컬럼 추가
concat_select['school_mean_2km']  = mean_achv     #반경 2km 내 평균 학업성취도 (모든 학교를 똑같이 취급한 평균 성취도; “인근 전반의 학업 수준”)
concat_select['school_max_2km']   = max_achv      #반경 2km 내 최고 학업성취도 (해당 지역에서 가장 성적이 좋은 한 학교의 성취도) 
concat_select['school_cnt_2km']   = count_schools #반경 2km 내 학교 개수 (학교 밀집도; 밀집도 자체가 학군 선호도의 간접 지표일 수도 있다는 가정)
concat_select['school_wmean_2km'] = wmean_achv    #거리 가중 학업성취도 평균 (가까운 학교에 더 높은 가중치를 준 평균)

# 8) 결과 확인
print(concat_select[['school_mean_2km','school_max_2km',
                     'school_cnt_2km','school_wmean_2km']].head())

   school_mean_2km  school_max_2km  school_cnt_2km  school_wmean_2km
0            89.92            97.1               5         89.708231
1            89.92            97.1               5         89.708231
2            89.92            97.1               5         89.708231
3            89.92            97.1               5         89.708231
4            89.92            97.1               5         89.708231


반경 2km 내 학교가 없는 경우, 평균 및 최고 학업성취도가 NaN 값으로 출력됨으로 이부분 처리가 필요함

추후 강남역, 버스 및 지하철과의 근접도를 고려하는 피쳐도 추가할 예정입니다.

In [142]:
concat_select.columns

Index(['번지', '본번', '부번', '아파트명', '전용면적', '계약일', '층', '건축년도', '도로명',
       'k-단지분류(아파트,주상복합등등)', 'k-전화번호', 'k-팩스번호', 'k-세대타입(분양형태)', 'k-관리방식',
       'k-복도유형', 'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사',
       'k-사용검사일-사용승인일', 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-수정일자', '고용보험관리번호',
       '경비비관리형태', '세대전기계약방법', '청소비관리형태', '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4',
       '단지승인일', '사용허가여부', '관리비 업로드', '좌표X', '좌표Y', '단지신청일', 'target',
       'is_test', '구', '동', '강남여부', '계약년', '계약월', '거래개월수', '건축연수',
       'Lot_Mst_within_1km', 'Mc_KFC_BK_within_1km', 'school_mean_2km',
       'school_max_2km', 'school_cnt_2km', 'school_wmean_2km'],
      dtype='object')

In [150]:
### 모델 학습에 활용할 컬럼만 남깁니다.

# 1) 유지할 컬럼 리스트
keep_cols = [
    '계약년',
    '계약월',
    '거래개월수',
    '전용면적',
    '구',
    '동',
    '도로명',
    '강남여부',
    '아파트명',
    '건축연수',
    'Lot_Mst_within_1km',
    'Mc_KFC_BK_within_1km',
    'school_mean_2km',
    'school_max_2km',
    'school_cnt_2km',
    'school_wmean_2km',
    'is_test',
    'target'
    ]

# 2) 선택
df_selected = concat_select[keep_cols].copy()

# 3) 확인
df_selected.head()

,계약년,계약월,거래개월수,전용면적,구,동,도로명,강남여부,아파트명,건축연수,Lot_Mst_within_1km,Mc_KFC_BK_within_1km,school_mean_2km,school_max_2km,school_cnt_2km,school_wmean_2km,is_test,target
0,2017,12,131,79.97,강남구,개포동,언주로 3,1,개포6차우성,38,0,0,89.92,97.1,5,89.708231,0,124000.0
1,2017,12,131,79.97,강남구,개포동,언주로 3,1,개포6차우성,38,0,0,89.92,97.1,5,89.708231,0,123500.0
2,2017,12,131,54.98,강남구,개포동,언주로 3,1,개포6차우성,38,0,0,89.92,97.1,5,89.708231,0,91500.0
3,2018,1,132,79.97,강남구,개포동,언주로 3,1,개포6차우성,38,0,0,89.92,97.1,5,89.708231,0,130000.0
4,2018,1,132,79.97,강남구,개포동,언주로 3,1,개포6차우성,38,0,0,89.92,97.1,5,89.708231,0,117000.0


In [151]:
### 모델 학습을 위해 학습 데이터와 테스트 데이터를 분할합니다.

dt_train = df_selected.query('is_test==0')
dt_test = df_selected.query('is_test==1')

In [ ]:
### 인코딩 및 학습을 실시합니다.

# 1) 컬럼 지정
num_cols = [
    '계약년','계약월','거래개월수','전용면적','건축연수',
    'Lot_Mst_within_1km','Mc_KFC_BK_within_1km',
    'school_mean_2km','school_max_2km',
    'school_cnt_2km','school_wmean_2km','강남여부'
]
cat_cols = ['구','동','도로명']

# 2) 전처리기 정의
preprocessor = ColumnTransformer([
    ("ord", OrdinalEncoder(
                   handle_unknown="use_encoded_value",
                   unknown_value=-1
    ), cat_cols),
], remainder="passthrough")

# 3) 파이프라인 구성
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(n_estimators=100, random_state=42))
])

# 4) 시계열 교차검증 세팅
tscv = TimeSeriesSplit(n_splits=5)

# 5) cross_val_score로 평가
X = concat_select[num_cols + cat_cols]   # 학습용 전체 데이터
y_log = concat_select["target_log"]

scores = cross_val_score(
    pipeline,
    X, y_log,
    cv=tscv,
    scoring="neg_mean_squared_error",
    n_jobs=-1
)

print("각 fold MSE:", -scores)
print("평균 RMSE:", np.sqrt(-scores).mean())

# 실제 단위(RMSE)로 변환해서 평가
real_rmse_list = []
for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y_log.iloc[train_idx], y_log.iloc[test_idx]
    pipeline.fit(X_train, y_train)
    y_pred_log = pipeline.predict(X_test)
    # log -> 실제 단위로 변환
    y_pred = np.expm1(y_pred_log)
    y_true = np.expm1(y_test)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    real_rmse_list.append(rmse)
print("각 fold 실제 RMSE:", real_rmse_list)
print("평균 실제 RMSE:", np.mean(real_rmse_list))

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py", line 345, in fit
    X, y = self._validate_data(
  File "/opt/conda/lib/python3.10/site-packages/sklearn/base.py", line 584, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/utils/validati

In [ ]:
# Target Log Transformation 적용
print("Log transformation 전 target 분포:")
print(df_selected['target'].describe())
print(f"Skewness: {df_selected['target'].skew():.3f}")

df_selected['target_log'] = np.log1p(df_selected['target'])

print("\nLog transformation 후 target 분포:")
print(df_selected['target_log'].describe())
print(f"Skewness: {df_selected['target_log'].skew():.3f}")